### 参数设置

In [8]:
case='1'
N=10 # 节点数量
F=5 # 无人机数量
M=20 # 任务数量

### 第一步节点生成

In [9]:
# 生成一个N个节点的(有向)图，每个节点的参数为（x,y)，其中x,y均为0-100之间的随机数
# 生成的图保存为1/graph.txt文件，每行为一个节点的信息，格式为：节点id x坐标 y坐标
import random
import os

def generate_graph(case,N,random_seed=0):
    if not os.path.exists(case):
        os.mkdir(case)
    graph_N={}
    random.seed(random_seed)
    with open(case+'/graph.txt', 'w') as f:
        f.write(f'{N}\n')
        for i in range(N):
            x = random.randint(0, 100)
            y = random.randint(0, 100)
            graph_N[i] = (x, y)
            f.write(f'{i} {x} {y}\n')
    return graph_N


graph_N=generate_graph(case,N,random_seed=0)
print('Done')

Done


### 第二步，无人机初始位置生成

In [10]:
## 生成一个F个无人机的初始位置，每个无人机在一个节点上，每个无人机的初始位置可以相同
## 生成的初始位置保存为1/initial.txt文件，每行为一个无人机的初始位置，格式为：无人机id 节点id
def generate_initial(case,F,N,random_seed=0):
    random.seed(random_seed)
    if not os.path.exists(case):
        os.mkdir(case)
    initial_pos = {}
    with open(case+'/initial_Pos.txt', 'w') as f:
        f.write(f'{F}\n')
        for i in range(F):
            pos = random.randint(0, N-1)
            initial_pos[i] = pos
            f.write(f'{i} {i}\n'.format(i,pos))
    return initial_pos

initial_pos = generate_initial(case,F,N,random_seed=0)


### 第三步生成M个任务，每个任务的参数为（x,y,T）,表示从x节点到y节点的任务，T为任务的截止时间

In [11]:
# ### 第三步生成M个任务，每个任务的参数为（x,y,T),表示从x节点到y节点的任务，T为任务的截止时间
# ### 生成的任务保存为1/task.txt文件，每行为一个任务的信息，格式为：任务id 起始节点id 终止节点id 截止时间
# def generate_task(case,M,N,random_seed=0):
#     random.seed(random_seed)
#     if not os.path.exists(case):
#         os.mkdir(case)
#     with open(case+'/task.txt', 'w') as f:
#         f.write(f'{M}\n')
#         for i in range(M):
#             x = random.randint(0, N-1)
#             y = random.randint(0, N-1)
#             T = random.randint(0, 100)
#             f.write(f'{i} {x} {y} {T}\n')
#     return

####  计算节点之间的距离矩阵

In [12]:
### 计算节点之间的距离，保存为1/distance.txt文件，每行为两个节点之间的距离，格式为：节点id1 节点id2 距离
def generate_distance(case,N,graph_N):
    if not os.path.exists(case):
        os.mkdir(case)
    distance_M = {}
    with open(case+'/distance.txt', 'w') as f:
        f.write(f'{N**2}\n')
        for i in range(N):
            for j in range(N):
                # if i==j:
                #     continue
                x1,y1 = graph_N[i]
                x2,y2 = graph_N[j]
                distance = ((x1-x2)**2+(y1-y2)**2)**0.5
                distance_M[(i,j)] = distance
                f.write(f'{i} {j} {distance}\n')
    return distance_M

distance_M = generate_distance(case,N,graph_N)

####  为每个无人机较为均匀的分配任务数量

In [13]:
### 为无人机分配均匀的分配任务数量
def generate_task_num(case,F,M,random_seed=0):
    random.seed(random_seed)
    task_num_M = {}
    task_mean=M//F
    for i in range(F):
        a=task_mean+random.randint(-task_mean//(F**2),task_mean//(F**2))
        task_num_M[i] = a
        M=M-a
        if F-i-1!=0:
            task_mean=M//(F-i-1)
    return task_num_M

task_num_M = generate_task_num(case,F,M,random_seed=0)

task_num_M

{0: 4, 1: 4, 2: 3, 3: 4, 4: 5}

#### 使用深度优先搜索来确定任务

In [14]:
def get_road(Task_N:int,N:int,graph_N:dict,distance_M:dict,pre:int,now:int,time:float,step:int,random_seed=0,P=0.5):
    if step>task_num_M[Task_N]:
        # print("Task_N  end",Task_N)
        return
    global Task_list
    if random_seed==0:
        random.seed()  

    # 从当前节点now出发，随机选择下一个节点
    next = random.randint(0, N-1)
    while next == now:
        next = random.randint(0, N-1)
    
    # 判断pre是不是0，0代表前驱不是任务线,下一次必为任务线
    if pre==0:
        end_time=int(time+distance_M[(now,next)]*random.uniform(0.98, 1.1))
        Task_list.append((Task_N,now,next,end_time))
        # print(Task_N,now,next,end_time)
        get_road(Task_N,N,graph_N,distance_M,1,next,end_time,step+1,P=P)
    else:
        # 随机生成一个0-1的数，如果大于0.5，则下一次必为任务线
        if random.uniform(0, 1)>P:
            end_time=int(time+distance_M[(now,next)]*random.uniform(0.98, 1.1))
            Task_list.append((Task_N,now,next,end_time))
            # print(Task_N,now,next,end_time)
            get_road(Task_N,N,graph_N,distance_M,1,next,end_time,step+1,P=P)
        else:
            end_time=int(time+distance_M[(now,next)]*random.uniform(0.98, 1.1))
            # print(Task_N,now,next,end_time)
            get_road(Task_N,N,graph_N,distance_M,0,next,end_time,step,P=P)
    return 



def generate_road(case,F,M,N,graph_N,distance_M,task_num_M,random_seed=0,P=0.5):
    if not os.path.exists(case):
        os.mkdir(case)
    global Task_list
    Task_list=[]

    for j in task_num_M.keys():
        get_road(j,N,graph_N,distance_M,pre=1,now=initial_pos[j],time=0,step=0,P=0.5)
    with open(case+'/road.txt', 'w') as f:
        f.write(f'{len(Task_list)}\n')
        for i in range(len(Task_list)):
            f.write(f'{i} {Task_list[i][1]}  {Task_list[i][2]} {Task_list[i][3]}\n')
    return Task_list

Task_list={}
generate_road(case,F,M,N,graph_N,distance_M,task_num_M,random_seed=0,P=0.5)
Task_list

[(0, 0, 5, 135),
 (0, 0, 1, 287),
 (0, 2, 3, 387),
 (0, 6, 8, 454),
 (0, 8, 0, 537),
 (1, 6, 8, 41),
 (1, 8, 6, 84),
 (1, 6, 1, 116),
 (1, 3, 7, 196),
 (1, 8, 7, 252),
 (2, 0, 6, 80),
 (2, 6, 8, 122),
 (2, 3, 5, 174),
 (2, 5, 2, 209),
 (3, 3, 0, 91),
 (3, 0, 6, 172),
 (3, 1, 4, 262),
 (3, 8, 4, 401),
 (3, 4, 9, 426),
 (4, 9, 6, 85),
 (4, 9, 4, 139),
 (4, 4, 8, 212),
 (4, 8, 7, 242),
 (4, 7, 9, 276),
 (4, 9, 8, 340)]